#### 할로윈 투자 전략
1. 11월 첫날에 매수 -> 4월 마지막날에 매도
2. 11월 첫날의 가격의 시가로 매수 
3. 4월 말일에 종가로 매도 
4. 수익율 계산 -> 매도가 / 매수가
5. 누적수익율 계산

In [ ]:
from datetime import datetime 
import pandas as pd
from dateutil.relativedelta import relativedelta

In [ ]:
# 데이터 로드 
df = pd.read_csv("../../csv/AMZN.csv", index_col='Date')

In [ ]:
df.head(1)

In [ ]:
# 인덱스를 시계열 데이터로 변경 
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
# 2000년 11월 데이터를 출력하려면?
df.loc['2000-11',].iloc[0,0]

In [ ]:
df.loc['2000-11',].head(1)

In [ ]:
df.loc['2001-04',].tail(1)

In [ ]:
df.loc['2000-11', 'Open'][0]

In [ ]:
df.loc['2001-04','Close'][-1]

In [ ]:
## 2000년부터 2010년까지 할로윈 투자 데이터를 추출 

# 비어있는 데이터프레임을 생성 
price_df = pd.DataFrame()

for i in range(2000, 2010, 1):
    # 구매하는 달을 문자열로 변수 생성
    buy_mon = f"{i}-11"             ## 2000-11, 2001-11, 2002-11, ...
    sell_mon = str(i+1) + "-04"     ## 2001-04, 2002-04, 2003-04, ...
    # buy_mon에 조건식이 참인 데이터에서 첫날의 데이터만 추출
    buy = df.loc[buy_mon, ].head(1)
    sell = df.loc[sell_mon, ].tail(1)
    # price_df와 buy를 결합 ( 단순한 행 결합 -> 유니언 결합 )
    # concat( {결합한 데이터프레임들을 리스트}, axis={0|1}, ignore_index={bool} )
    price_df = pd.concat([price_df, buy, sell], axis=0)

In [ ]:
price_df = price_df[['Open', 'Close']]

In [ ]:
# 수익율 컬럼을 생성해서 1을 대입 
price_df['rtn'] = 1

In [ ]:
# 위치의 값이 홀수인 경우 수익율을 계산
for i in range(len(price_df)):
    # 홀수인 조건 
    if i % 2 == 1:
        rtn = price_df.iloc[i,]['Close'] / price_df.iloc[i-1, ]['Open']
        price_df.iloc[i,2] = rtn

In [ ]:
price_df

In [ ]:
price_df['rtn2'] = 1

In [ ]:
for i in range(1, len(price_df), 2):
    rtn = price_df.iloc[i,]['Close'] / price_df.iloc[i-1,]['Open']
    price_df.iloc[i, 3] = rtn

In [ ]:
price_df

In [ ]:
price_df['rtn3'] = 1

In [ ]:
# 데이터중 4월의 데이터만 추출 
flag = price_df.index.month == 4

In [ ]:
for i in price_df.index:
    # i가 의미하는것은? -> 인덱스 데이터
    if i.month == 4:
        rtn = price_df.loc[i, 'Close'] / price_df.shift().loc[i, 'Open']
        price_df.loc[i, 'rtn3'] = rtn


In [ ]:
price_df

In [ ]:
price_df['rtn'].cumprod()

In [ ]:
start = datetime(year=2000, month=11, day=1)
start

In [ ]:
# 시작일로부터 5개월 뒤 
end = start + relativedelta(months=5)

In [ ]:
start = start.strftime("%Y-%m")
start

In [ ]:
end = end.strftime('%Y-%m')
end

### 할로윈 투자 전략을 함수화
- 매개변수 4개 : 데이터프레임, 시작년도, 종료년도, 시작월
    - 시작년도는 기본값이 2000
    - 종료년도는 기본값이 2010
    - 시작월은 기본값이 11
- 컬럼에 Date가 존재하는가?
    - 존재한다면 Date를 인덱스로 변경
- index를 시계열 데이터로 변환 

In [ ]:
## 할로윈 전략을 함수화 
def six_month(_df, 
              _start = 2000, 
              _end = datetime.now().year, 
              _month = 11):
    df = _df.copy()
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True) 
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    df = df[['Open', 'Close']]
    # 비어있는 데이터프레임 생성
    result = pd.DataFrame()
    for i in range(_start, _end):
        start = datetime(year = i, month=_month, day=1)
        end = start + relativedelta(months=5)
        buy_mon = start.strftime('%Y-%m')
        sell_mon = end.strftime('%Y-%m')
        try:
            buy = df.loc[buy_mon,].head(1)
            sell = df.loc[sell_mon, ].tail(1)
            
            result = pd.concat([result, buy, sell], axis=0)
        except:
            break
    # 수익율 계산
    result['rtn'] = 1
    for i in range(1,len(result),2):
        rtn = result.iloc[i,]['Close'] / result.iloc[i-1,]['Open']
        result.iloc[i, 2] = rtn
    # 누적 수익율을 계산
    result['acc_rtn'] = result['rtn'].cumprod()
    # 총 누적수익율을 변수에 저장
    acc_rtn = result.iloc[-1,]['acc_rtn']
    return result, acc_rtn


In [ ]:
df = pd.read_csv('../../csv/AMZN.csv')

In [ ]:
six_month(df)

In [ ]:
# 야후파이낸스 라이브러리 설치 
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
aapl = yf.download('AAPL', start='2020-01-01')

In [ ]:
aapl.head()

In [ ]:
sdi = yf.download('006400.KS', start='2015-01-01')

In [ ]:
sdi.head()

In [ ]:
six_month(aapl, _start=2020)

In [59]:
# 1월부터 12월까지 6개월 유지 투자 전략
for i in range(1, 13):
    df, acc_rtn = six_month(sdi, _start=2015, _month=i)
    print(f"{i}월에 투자 시작 시 수익율 : {round(acc_rtn, 2)}")

C:\Users\moons\AppData\Local\Temp\ipykernel_8208\4140695437.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.961038961038961' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[i, 2] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_8208\4140695437.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.6651162790697674' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[i, 2] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_8208\4140695437.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.6123188405797102' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[i, 2] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_820

1월에 투자 시작 시 수익율 : 2.42
2월에 투자 시작 시 수익율 : 1.95
3월에 투자 시작 시 수익율 : 2.54
4월에 투자 시작 시 수익율 : 1.79
5월에 투자 시작 시 수익율 : 1.81
6월에 투자 시작 시 수익율 : 1.6
7월에 투자 시작 시 수익율 : 1.5
8월에 투자 시작 시 수익율 : 1.5
9월에 투자 시작 시 수익율 : 1.11
10월에 투자 시작 시 수익율 : 1.91
11월에 투자 시작 시 수익율 : 1.8
12월에 투자 시작 시 수익율 : 2.3


C:\Users\moons\AppData\Local\Temp\ipykernel_8208\4140695437.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0555555555555556' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[i, 2] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_8208\4140695437.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9336099585062241' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[i, 2] = rtn
